In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.utils import to_categorical
import xgboost as xgb
from sklearn.metrics import accuracy_score

2024-06-14 21:20:06.882671: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-14 21:20:07.005513: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-14 21:20:07.326093: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-14 21:20:09.006434: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# Load the CSV files

train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
validation_df = pd.read_csv('val.csv')

# Separate features and target
X_train = train_df.drop(columns=['anomaly'])
y_train = train_df['anomaly']
X_test = test_df.drop(columns=['anomaly'])
y_test = test_df['anomaly']
X_validation = validation_df.drop(columns=['anomaly'])
y_validation = validation_df['anomaly']

# Columns
categorical_columns = ["Material number", "Supplier", "Contract", "Contract Position", "Procurement type", 
                       "Special procurement type", "Dispatcher", "Buyer", "Purchasing group", 
                       "Purchasing lot size", "Calendar", "Plant", "Plant information record", 
                       "Information record number", "Information record type", "Product group",
                       "Base unit"]
numerical_columns = ["Fulfillment time", "Fixed contract 1", "Fixed contract 2", "Total quantity", "Total value", 
                     "Price unit", "Plant processing time", "Material master time"]


In [3]:
# Preprocess the Data
# Normalize Numerical Features
# Use Autoencoder for Categorical Features

scaler = StandardScaler()
X_train_num = scaler.fit_transform(X_train[numerical_columns])
X_test_num = scaler.transform(X_test[numerical_columns])
X_validation_num = scaler.transform(X_validation[numerical_columns])



# Train an autoencoder for the categorical features
# Convert categorical columns to strings for encoding

X_train_cat = X_train[categorical_columns].astype(str)
X_test_cat = X_test[categorical_columns].astype(str)
X_validation_cat = X_validation[categorical_columns].astype(str)

# Simple one-hot encoding before feeding to autoencoder
X_train_cat_enc = pd.get_dummies(X_train_cat)
X_test_cat_enc = pd.get_dummies(X_test_cat)
X_validation_cat_enc = pd.get_dummies(X_validation_cat)

# Align the columns in case some categories are missing in some sets
X_train_cat_enc, X_test_cat_enc = X_train_cat_enc.align(X_test_cat_enc, fill_value=0, axis=1)
X_train_cat_enc, X_validation_cat_enc = X_train_cat_enc.align(X_validation_cat_enc, fill_value=0, axis=1)

# Autoencoder structure
input_dim = X_train_cat_enc.shape[1]
encoding_dim = 10  # Dimension of the encoded representation

input_layer = Input(shape=(input_dim,))
encoder = Dense(encoding_dim, activation="relu")(input_layer)
decoder = Dense(input_dim, activation="sigmoid")(encoder)
autoencoder = Model(inputs=input_layer, outputs=decoder)
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

# Train the autoencoder
autoencoder.fit(X_train_cat_enc, X_train_cat_enc, 
                epochs=50, 
                batch_size=256, 
                shuffle=True, 
                validation_data=(X_validation_cat_enc, X_validation_cat_enc))

# Get the encoder part
encoder_model = Model(inputs=input_layer, outputs=encoder)

# Encode the categorical features
X_train_cat_enc = encoder_model.predict(X_train_cat_enc)
X_test_cat_enc = encoder_model.predict(X_test_cat_enc)
X_validation_cat_enc = encoder_model.predict(X_validation_cat_enc)


MemoryError: Unable to allocate 15.9 GiB for an array with shape (158742, 107391) and data type bool

In [ ]:
# Combine the Processed Features
# Combine the numerical and encoded categorical features
X_train_final = np.hstack((X_train_num, X_train_cat_enc))
X_test_final = np.hstack((X_test_num, X_test_cat_enc))
X_validation_final = np.hstack((X_validation_num, X_validation_cat_enc))


In [ ]:
# Train the XGBoost Model
# Initialize the model
model = xgb.XGBClassifier()

# Train the model
model.fit(X_train_final, y_train)

# Predict and evaluate
y_pred = model.predict(X_test_final)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")



In [ ]:
# Validate the Model
y_val_pred = model.predict(X_validation_final)
val_accuracy = accuracy_score(y_validation, y_val_pred)
print(f"Validation Accuracy: {val_accuracy}")


In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.optimizers import Adam

# Step 1: Load the Dataset
file_path = 'supervised_dataset_chunk_part_1.csv'
data = pd.read_csv(file_path)
print(data.shape)
# Step 2: Define Categorical and Numerical Columns
categorical_columns = ["Material number", "Supplier", "Contract", "Contract Position", "Procurement type", 
                       "Special procurement type", "Dispatcher", "Buyer", "Purchasing group", 
                       "Purchasing lot size", "Calendar", "Plant", "Plant information record", 
                       "Information record number", "Information record type",  "Product group",
                       "Base unit"]
numerical_columns = ["Fulfillment time", "Fixed contract 1", "Fixed contract 2", "Total quantity", "Total value", 
                     "Price unit", "Plant processing time", "Material master time"]

categorical_data = data[categorical_columns]
categorical_data_encoded = pd.get_dummies(categorical_data).astype('float32')
input_dim = categorical_data_encoded.shape[1]

### Step 4: Define and Train the Autoencoder
encoding_dim = 32  # Dimension of the encoded representation
autoencoder = Sequential([
    Dense(encoding_dim, input_shape=(input_dim,), activation='relu'),
    Dense(input_dim, activation='sigmoid')
])
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')
autoencoder.fit(categorical_data_encoded.values, categorical_data_encoded.values, epochs=20, batch_size=256, shuffle=True, validation_split=0.1)


# Step 5: Encode Categorical Data
encoder = Model(inputs=autoencoder.input, outputs=autoencoder.layers[0].output)
encoded_categorical_data = encoder.predict(categorical_data_encoded)

# Step 6: Combine Encoded Categorical Data with Numerical Data
data_combined = np.concatenate((data[numerical_columns].values, encoded_categorical_data), axis=1)

# Step 7: Define Features and Target
X = data_combined
y = data['anomaly']

# Step 8: Train XGBoost Model
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
xgb_model = xgb.XGBClassifier(objective='binary:logistic', random_state=42)
xgb_model.fit(X_train, y_train)

# Step 9: Identify Anomalies and Top 8 Features
y_pred = xgb_model.predict(X_test)
# Ensure we are only summing numerical columns when counting anomalies
anomalies_indices = np.where(y_pred == 1)[0]  # Indices of detected anomalies

# Filter data to include only numerical columns for anomaly summing
numerical_data = data[numerical_columns]  # Assuming 'numerical_columns' is predefined list of your numeric columns

# Count anomalies in each numerical column
anomalies_count = {col: np.sum(numerical_data.iloc[anomalies_indices][col]) for col in numerical_columns}  # Count anomalies in each column

# Get the top 8 features with the most anomalies
top_8_anomaly_features = sorted(anomalies_count, key=anomalies_count.get, reverse=True)[:8]


for feature in top_8_anomaly_features:
    plt.figure(figsize=(10, 6))
    # Drop NaN values from the column
    feature_data = data[feature].dropna()
    if feature_data.empty:
        print(f"No data available to plot for {feature}.")
    else:
        plt.hist(feature_data, bins=50, label=f'Distribution of {feature}')
        plt.title(f'Distribution of {feature}')
        plt.xlabel(feature)
        plt.ylabel('Frequency')
        plt.legend()
        plt.show()



2024-06-10 21:28:19.182465: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-10 21:28:19.216081: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-10 21:28:19.316370: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-10 21:28:20.397632: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


(10000, 26)


/home/gravityfall_kevin/.conda/envs/ai/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2024-06-10 21:28:24.898606: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 360396000 exceeds 10% of free system memory.
2024-06-10 21:28:27.053528: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 360396000 exceeds 10% of free system memory.


Epoch 1/20
35/36 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step - loss: nan

2024-06-10 21:28:31.474752: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 40044000 exceeds 10% of free system memory.
2024-06-10 21:28:31.603436: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 40044000 exceeds 10% of free system memory.


36/36 ━━━━━━━━━━━━━━━━━━━━ 4s 80ms/step - loss: nan - val_loss: nan
Epoch 2/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 3s 72ms/step - loss: nan - val_loss: nan
Epoch 3/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 5s 68ms/step - loss: nan - val_loss: nan
Epoch 4/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 3s 70ms/step - loss: nan - val_loss: nan
Epoch 5/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 2s 68ms/step - loss: nan - val_loss: nan
Epoch 6/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 2s 68ms/step - loss: nan - val_loss: nan
Epoch 7/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 2s 66ms/step - loss: nan - val_loss: nan
Epoch 8/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 2s 65ms/step - loss: nan - val_loss: nan
Epoch 9/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 2s 67ms/step - loss: nan - val_loss: nan
Epoch 10/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 2s 67ms/step - loss: nan - val_loss: nan
Epoch 11/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 3s 73ms/step - loss: nan - val_loss: nan
Epoch 12/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 3s 74ms/step - loss: nan - val_loss: nan
Epoch 13/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 3s 91ms/step - loss: 

ValueError: The layer sequential has never been called and thus has no defined input.